### Імпортування бібліотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import netCDF4 as nc
from sklearn.linear_model import LinearRegression
import datetime 

2024-06-14 10:49:35.371750: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 10:49:35.400732: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 10:49:35.400762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 10:49:35.401780: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-14 10:49:35.409853: I tensorflow/core/platform/cpu_feature_guar

### Перевірка застосування графічного процесора

In [2]:
#check if GPU is available
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-06-14 10:49:37.214920: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-14 10:49:37.256732: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-14 10:49:37.256783: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


### Завантажження данних у словник

In [3]:

data = {
    year: {
            "climate": nc.Dataset("data/train/" + year + "/climate.nc"),
            "polution": nc.Dataset("data/train/" + year + "/polution.nc"),
        }
    for year in ["2021", "2022", "2023"]
}

In [4]:
test = {
        "climate": nc.Dataset("data/test/" + "/climate.nc"),
        "polution": nc.Dataset("data/test/" + "/polution.nc"),
}

#### Кількість географічних точок на кліматичних данних і на данних про викидів різниться. 
#### Знаходжу спільні точки

In [5]:
pol_lat_2021 = np.array(data["2021"]["polution"].variables["latitude"][: :-1])
climate_lat_2021 = np.array(data["2021"]["climate"].variables['lat'][:])
sorter = np.argsort(pol_lat_2021)
lats = -1 - np.searchsorted(pol_lat_2021, climate_lat_2021, sorter=sorter)


pol_lon_2021 = np.array(data["2021"]["polution"].variables["longitude"][:])
climate_lon_2021 = np.array(data["2021"]["climate"].variables['lon'][:])
sorter = np.argsort(pol_lon_2021)
lons = np.searchsorted(pol_lon_2021, climate_lon_2021, sorter=sorter)

### Завантаження данних у множину датафреймів

Одразу позбуваємось даних про вітер на висоті 50 метрів

In [6]:
start_date = datetime.date(2021, 1, 1); end_date = datetime.date(2023, 12, 31)
time = [start_date + datetime.timedelta(days=i) for i in range((end_date - start_date).days + 1)]

train_data = {}

atributes_climate = list(data["2021"]["climate"].variables.keys())[:-3]
atributes_climate.remove("WS50M")
atributes_climate.remove("WD50M")
atributes_polution = list(data["2021"]["polution"].variables.keys())[4:]

for i in range(len(lats)):
    for j in range(len(lons)):

        i_p = lats[i]
        j_p = lons[j]

        _data = {
            atribute: np.array(data["2021"]["climate"].variables[atribute][:, i, j])
            for atribute in atributes_climate
        }

        for atribute in atributes_polution:
            _data[atribute] = np.array(data["2021"]["polution"].variables[atribute][:, 0, i_p, j_p])


        _data["time"] = time

        for year in ["2022", "2023"]:
            for atribute in atributes_climate:
                _data[atribute] = np.append(_data[atribute], np.array(data[year]["climate"].variables[atribute][:, 0, 0]))
            for atribute in atributes_polution:
                _data[atribute] = np.append(_data[atribute], np.array(data[year]["polution"].variables[atribute][:, 0, 0, -1]))
        
        train_data[str(float(data["2021"]["climate"].variables["lat"][i])) + "x" + str(float(data["2021"]["climate"].variables["lon"][j]))]= pd.DataFrame(_data)


In [7]:
test_data = {}
start_date = datetime.date(2024, 1, 1); end_date = datetime.date(2024, 3, 31)
time = [start_date + datetime.timedelta(days=i) for i in range((end_date - start_date).days + 1)]

for i in range(len(lats)):
    for j in range(len(lons)):

        i_p = lats[i]
        j_p = lons[j]

        _data = {
            atribute: np.array(test["climate"].variables[atribute][:, i, j])
            for atribute in atributes_climate
        }

        for atribute in atributes_polution:
            _data[atribute] = np.array(test["polution"].variables[atribute][:, 0, i_p, j_p])


        _data["time"] = time

        test_data[str(float(data["2021"]["climate"].variables["lat"][i])) + "x" + str(float(data["2021"]["climate"].variables["lon"][j]))]= pd.DataFrame(_data)


In [8]:
def norm(smth):
    smth = smth - np.min(smth)
    smth = smth/np.max(smth)
    return smth

def log_norm(smth):
    smth = np.log(1 + smth)
    return norm(smth)

In [9]:
sample = train_data["44.25x30.75"].copy()
sample_test = test_data["44.25x30.75"].copy()

In [10]:
sample["co_conc"] = log_norm(sample["co_conc"])
sample["pm10_conc"] = log_norm(sample["pm10_conc"])
sample["pm2p5_conc"] = log_norm(sample["pm2p5_conc"])
sample["o3_conc"] = norm(sample["o3_conc"])

sample_test["co_conc"] = log_norm(sample_test["co_conc"])
sample_test["pm10_conc"] = log_norm(sample_test["pm10_conc"])
sample_test["pm2p5_conc"] = log_norm(sample_test["pm2p5_conc"])
sample_test["o3_conc"] = norm(sample_test["o3_conc"])


for i in atributes_climate:
    sample[i] = log_norm(sample[i])
    sample_test[i] = log_norm(sample_test[i])
sample.head()

,ALLSKY_SFC_SW_DWN,WD10M,QV2M,PS,WS10M,T2M,co_conc,no2_conc,no_conc,o3_conc,pm10_conc,pm2p5_conc,so2_conc,time
0,0.269043,0.885004,0.454919,0.567388,0.610355,0.694195,0.297221,2.526610,0.559673,0.375551,0.535488,0.510513,1.706610,2021-01-01
1,0.449505,0.822524,0.332772,0.760872,0.449125,0.664428,0.398022,0.953106,0.170985,0.469237,0.469305,0.473638,2.275357,2021-01-02
2,0.358171,0.699968,0.439985,0.742353,0.667446,0.673773,0.297877,0.590444,0.064119,0.606194,0.676444,0.466534,0.888747,2021-01-03
3,0.190263,0.706666,0.477366,0.651348,0.548200,0.671218,0.287901,0.792554,0.099135,0.612410,0.756116,0.703906,1.191523,2021-01-04
4,0.208614,0.810651,0.474516,0.548686,0.624359,0.687206,0.344157,2.439456,0.288225,0.497168,0.730189,0.746955,1.623917,2021-01-05


In [11]:
params = sample.get(atributes_climate)
params_test = sample_test.get(atributes_climate)

co_conc = np.array(sample.get("co_conc"))
pm10_conc = np.array(sample.get("pm10_conc"))
pm2p5_conc = np.array(sample.get("pm2p5_conc"))
o3_conc = np.array(sample.get("o3_conc"))

co_conc_test = np.array(sample_test.get("co_conc"))
pm10_conc_test = np.array(sample_test.get("pm10_conc"))
pm2p5_conc_test = np.array(sample_test.get("pm2p5_conc"))
o3_conc_test = np.array(sample_test.get("o3_conc"))

period = 1
for i in range(period):
    co_conc[i] = sample["co_conc"][0]
    pm10_conc[i] = sample["pm10_conc"][0]
    pm2p5_conc[i] = sample["pm2p5_conc"][0]
    o3_conc[i] = sample["o3_conc"][0]


    co_conc_test[i] = sample_test["co_conc"][0]
    pm10_conc_test[i] = sample_test["pm10_conc"][0]
    pm2p5_conc_test[i] = sample_test["pm2p5_conc"][0]
    o3_conc_test[i] = sample_test["o3_conc"][0]


for i in range(len(co_conc) - period):
    co_conc[i + period] = sample["co_conc"][i]
    pm10_conc[i + period] = sample["pm10_conc"][i]
    pm2p5_conc[i + period] = sample["pm2p5_conc"][i]
    o3_conc[i + period] = sample["o3_conc"][i]

for i in range(len(co_conc_test) - period):
    co_conc_test[i + period] = sample_test["co_conc"][i]
    pm10_conc_test[i + period] = sample_test["pm10_conc"][i]
    pm2p5_conc_test[i + period] = sample_test["pm2p5_conc"][i]
    o3_conc_test[i + period] = sample_test["o3_conc"][i]

params["co_conc"] = co_conc
params["pm10_conc"] = pm10_conc
params["pm2p5_conc"] = pm2p5_conc
params["o3_conc"] = o3_conc


params_test["co_conc"] = co_conc_test
params_test["pm10_conc"] = pm10_conc_test
params_test["pm2p5_conc"] = pm2p5_conc_test
params_test["o3_conc"] = o3_conc_test

params

/tmp/ipykernel_52980/3103814991.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  params["co_conc"] = co_conc
/tmp/ipykernel_52980/3103814991.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  params["pm10_conc"] = pm10_conc
/tmp/ipykernel_52980/3103814991.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,ALLSKY_SFC_SW_DWN,WD10M,QV2M,PS,WS10M,T2M,co_conc,pm10_conc,pm2p5_conc,o3_conc
0,0.269043,0.885004,0.454919,0.567388,0.610355,0.694195,0.297221,0.535488,0.510513,0.375551
1,0.449505,0.822524,0.332772,0.760872,0.449125,0.664428,0.297221,0.535488,0.510513,0.375551
2,0.358171,0.699968,0.439985,0.742353,0.667446,0.673773,0.398022,0.469305,0.473638,0.469237
3,0.190263,0.706666,0.477366,0.651348,0.548200,0.671218,0.297877,0.676444,0.466534,0.606194
4,0.208614,0.810651,0.474516,0.548686,0.624359,0.687206,0.287901,0.756116,0.703906,0.612410
...,...,...,...,...,...,...,...,...,...,...
1090,0.418688,0.907523,0.428538,0.643893,0.600576,0.671859,0.363551,0.199341,0.114184,0.485521
1091,0.382203,0.734391,0.301301,0.807091,0.499211,0.634175,0.369335,0.146640,0.156933,0.425808
1092,0.426988,0.898679,0.411348,0.666231,0.576074,0.650121,0.445790,0.347094,0.383669,0.414040
1093,0.318726,0.903820,0.387919,0.638311,0.417023,0.620460,0.468065,0.538743,0.590444,0.293514


In [12]:
result_test = sample_test.get(["co_conc", "pm10_conc", "pm2p5_conc", "o3_conc"])

In [13]:
result = sample.get(["co_conc", "pm10_conc", "pm2p5_conc", "o3_conc"])

In [14]:
model = LinearRegression().fit(params, result)
print("coeficients = ", model.coef_)
print("intercept = ", model.intercept_)
print("score = ", model.score(params, result))

coeficients =  [[-0.07277843  0.01966375 -0.01730353  0.02088133 -0.01367028 -0.02961228
   0.7141616   0.08657718 -0.11445416  0.05833074]
 [-0.05098315  0.01565439  0.20456742  0.0333698  -0.05788686 -0.25909585
   0.14588802  0.657844   -0.25067288  0.13360421]
 [-0.03625394  0.02564041  0.2554779   0.06010515 -0.1021494  -0.34026822
   0.17511114  0.23860456  0.1344353   0.12560147]
 [ 0.17135097 -0.05922762  0.20204099  0.01065647 -0.02673335 -0.31681752
   0.08519619  0.12023737 -0.09573757  0.623516  ]]
intercept =  [0.1234163  0.24246237 0.28314596 0.26592523]
score =  0.4586388700450863


In [15]:
prediction = np.array(model.predict(params))

In [16]:
for i, j in enumerate(["co_conc", "pm10_conc", "pm2p5_conc", "o3_conc"]):
    print(j)
    print("max difference = ", np.max(np.abs(prediction.T[i] - result[j])), "=", np.max(np.abs(prediction.T[i] - result[j])) / np.mean(result[j])*100, "% "
            "mean difference = " ,np.mean(np.abs(prediction.T[i] - result[j])), "=" , np.mean(np.abs(prediction.T[i] - result[j])) / np.mean(result[j])*100, "% ")

co_conc
max difference =  0.5121092796325684 = 164.32165378757105 % mean difference =  0.051783647 = 16.615934669971466 % 
pm10_conc
max difference =  0.6122204065322876 = 146.63766469400525 % mean difference =  0.10659134 = 25.53052008152008 % 
pm2p5_conc
max difference =  0.6742343902587891 = 158.19134707522014 % mean difference =  0.11712361 = 27.479970455169678 % 
o3_conc
max difference =  0.4476340711116791 = 75.06809057125511 % mean difference =  0.08102329 = 13.587580621242523 % 


In [17]:
for i, j in enumerate(["co_conc", "pm10_conc", "pm2p5_conc", "o3_conc"]):
    print(j)
    print("max difference = ", np.max(np.abs(params[j] - result[j])), "=", np.max(np.abs(params[j] - result[j])) / np.mean(result[j]) * 100, "% "
            "mean difference = " ,np.mean(np.abs(params[j] - result[j])), "=" , np.mean(np.abs(params[j] - result[j])) / np.mean(result[j]) * 100, "% ")

co_conc
max difference =  0.6539776921272278 = 209.8432896736964 % mean difference =  0.05392327 = 17.302481830120087 % 
pm10_conc
max difference =  0.8237142562866211 = 197.29419932469906 % mean difference =  0.12041095 = 28.84056270122528 % 
pm2p5_conc
max difference =  0.7831641435623169 = 183.74884556627526 % mean difference =  0.13648252 = 32.02202916145325 % 
o3_conc
max difference =  0.49222955107688904 = 82.54673829970687 % mean difference =  0.08908538 = 14.93958979845047 % 


In [18]:
prediction_test = np.array(model.predict(params_test))

In [19]:
for i, j in enumerate(["co_conc", "pm10_conc", "pm2p5_conc", "o3_conc"]):
    print(j)
    print("max difference = ", np.max(np.abs(prediction_test.T[i] - result_test[j])), "=", np.max(np.abs(prediction_test.T[i] - result_test[j])) / np.mean(result_test[j]) * 100, "% "
            "mean difference = " ,np.mean(np.abs(prediction_test.T[i] - result_test[j])), "=" , np.mean(np.abs(prediction_test.T[i] - result_test[j])) / np.mean(result_test[j]) * 100, "% ")

co_conc
max difference =  0.6230512857437134 = 125.86019324167658 % mean difference =  0.17632233 = 35.618194937705994 % 
pm10_conc
max difference =  0.5310489535331726 = 124.96907866628553 % mean difference =  0.1676483 = 39.451831579208374 % 
pm2p5_conc
max difference =  0.512850284576416 = 94.58180019806314 % mean difference =  0.16116373 = 29.72242832183838 % 
o3_conc
max difference =  0.4724677801132202 = 84.48348245256054 % mean difference =  0.13630809 = 24.37368780374527 % 


In [20]:
for i, j in enumerate(["co_conc", "pm10_conc", "pm2p5_conc", "o3_conc"]):
    print(j)
    print("max difference = ", np.max(np.abs(params_test[j] - result_test[j])), "=", np.max(np.abs(params_test[j] - result_test[j])) / np.mean(result_test[j]) * 100, "% "
            "mean difference = " ,np.mean(np.abs(params_test[j] - result_test[j])), "=" , np.mean(np.abs(params_test[j] - result_test[j])) / np.mean(result_test[j]) * 100, "% ")

co_conc
max difference =  0.6667367219924927 = 134.6849201524882 % mean difference =  0.18719749 = 37.815046310424805 % 
pm10_conc
max difference =  0.7093958854675293 = 166.93856494153349 % mean difference =  0.16320014 = 38.40506672859192 % 
pm2p5_conc
max difference =  0.5648395419120789 = 104.16985678621285 % mean difference =  0.1655819 = 30.53724467754364 % 
o3_conc
max difference =  0.5541915893554688 = 99.0967794744641 % mean difference =  0.16095145 = 28.780245780944824 % 


In [49]:
mas = ["CO2", "pm10", "pm2.5", "озон"]
for i, j in enumerate(["co_conc", "pm10_conc", "pm2p5_conc", "o3_conc"]):
    print(mas[i], " & {:6.2f} \% & {:6.2f} \% ".format(
np.mean(np.abs(prediction.T[i] - result[j])**2) / np.mean(result[j])**2 * 100,
np.mean(np.abs(params[j] - result[j])**2) / np.mean(result[j])**2 * 100,
#np.max(np.abs(prediction.T[i] - result[j])**2) / np.mean(result[j])**2 * 100,
#np.max(np.abs(params[j] - result[j])**2) / np.mean(result[j])**2 * 100
        ))

CO2  &   5.13 \% &   6.29 \% 
pm10  &  10.70 \% &  15.08 \% 
pm2.5  &  12.12 \% &  17.99 \% 
озон  &   2.94 \% &   3.72 \% 


In [48]:
mas = ["CO2", "pm10", "pm2.5", "озон"]
for i, j in enumerate(["co_conc", "pm10_conc", "pm2p5_conc", "o3_conc"]):
    print(mas[i], " & {:6.2f} \% & {:6.2f} \%".format(
np.mean(np.abs(prediction_test.T[i] - result_test[j])**2) / np.mean(result_test[j]**2) * 100,
np.mean(np.abs(params_test[j] - result_test[j])**2) / np.mean(result_test[j]**2) * 100
#np.max(np.abs(prediction_test.T[i] - result_test[j])**2) / np.mean(result_test[j]**2) * 100,
#np.max(np.abs(params_test[j] - result_test[j])**2) / np.mean(result_test[j]**2) * 100
        ))

CO2  &  16.52 \% &  20.04 \%
pm10  &  18.78 \% &  19.76 \%
pm2.5  &  12.50 \% &  14.55 \%
озон  &   8.27 \% &  12.04 \%


In [23]:
#plt.plot(prediction_test)
#plt.plot(result_test)